In [1]:
import pandas as pd
import numpy as np
import pickle

from PopSynthesis.Methods.connect_HH_PP.scripts.adjust_atts_by_counts import *
from PopSynthesis.Methods.connect_HH_PP.scripts.sample_hh_main import *
from PopSynthesis.Methods.connect_HH_PP.scripts.process_all_hh_pp import *
from PopSynthesis.Methods.connect_HH_PP.scripts.get_hh_main_combine import *
from PopSynthesis.Methods.connect_HH_PP.paras_dir import *

In [2]:
with open(os.path.join(processed_data, 'dict_pp_states.pickle'), 'rb') as handle:
    pp_state_names = pickle.load(handle)

all_rela_exist = ALL_RELA.copy()
all_rela_exist.remove("Self")
dict_model_inference = inference_model_get(all_rela_exist, pp_state_names)
dict_pool_sample = pools_get(all_rela_exist, dict_model_inference, int(1e7))

Learn BN Spouse


  0%|          | 0/1000000 [00:00<?, ?it/s]

Learn BN Child


  0%|          | 0/1000000 [00:00<?, ?it/s]

Learn BN Grandchild


...

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [3]:
rela = "Grandchild"
geo_lev = "POA"

In [4]:
with open(os.path.join(processed_data, 'dict_hh_states.pickle'), 'rb') as handle:
        hh_state_names = pickle.load(handle)
state_names = hh_state_names | pp_state_names

#learning to get the HH only with main person
df_seed_hh_main = pd.read_csv(os.path.join(processed_data, "connect_hh_main.csv"))
# drop all the ids as they are not needed for in BN learning
id_cols_hh_main = [x for x in df_seed_hh_main.columns if "hhid" in x or "persid" in x]
df_seed_hh_main = df_seed_hh_main.drop(columns=id_cols_hh_main)
logger.info("GETTING the pool HH and Main")
pool_hh_main = get_pool(df_seed_hh_main, state_names, int(1e7))
pool_hh_main = pool_hh_main.value_counts().reset_index()

2024-02-15 17:09:14,543 - INFO: GETTING the pool HH and Main


Learn BN


  0%|          | 0/1000000 [00:00<?, ?it/s]

Doing the sampling


  0%|          | 0/14 [00:00<?, ?it/s]

In [5]:
hh_df = pd.read_csv(os.path.join(output_dir, "adjust", "final", "saving_hh_dwelltype.csv"), low_memory=False)
if "hhid" not in hh_df.columns:
    hh_df["hhid"] = hh_df.index

# Sample hh main
logger.info("GETTING the main people")
combine_df_hh_main, del_hh = get_combine_df(hh_df, pool_hh_main)

2024-02-15 17:12:17,967 - INFO: GETTING the main people
2024-02-15 17:12:17,975 - INFO: PROCESSING the pool for later process
2024-02-15 17:12:19,019 - INFO: Process the given HH df
2024-02-15 17:12:38,085 - INFO: Start combining to connect hh and pp
2024-02-15 17:19:01,155 - INFO: Start select based on distribution in the pool
2024-02-15 17:21:19,002 - INFO: Final processing before outputting the syn hh-main connect


In [10]:
_, main_pp_df_all = process_combine_df(combine_df_hh_main)
main_pp_df_all[all_rela_exist] = main_pp_df_all[all_rela_exist].astype(float).astype(int)

C:\Users\dlaa0001\AppData\Local\Temp\ipykernel_776\3942609270.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_pp_df_all[all_rela_exist] = main_pp_df_all[all_rela_exist].astype(float).astype(int)


## Method 1, the old one, let's check

In [11]:
to_del_df, pop_rela_base = process_rela_fast(main_pp_df_all, rela, dict_pool_sample[rela])

PROCESS to keep ('50-59', 'M', '$1-199 p.w.', 'No Licence', 'N')
PROCESS to keep ('60-69', 'M', 'Zero Income', 'No Licence', 'N')
PROCESS to keep ('50-59', 'M', '$1500-1999 p.w.', 'No Licence', 'N')
PROCESS to keep ('50-59', 'M', '$300-399 p.w.', 'No Licence', 'N')
PROCESS to keep ('70-79', 'F', '$1500-1999 p.w.', 'No Licence', 'N')
PROCESS to keep ('30-39', 'F', '$1000-1249 p.w.', 'No Licence', 'N')
PROCESS to keep ('50-59', 'F', 'Zero Income', 'No Licence', 'Y')
PROCESS to keep ('0-9', 'M', '$1250-1499 p.w.', 'Some Licence', 'Y')
PROCESS to keep ('60-69', 'F', '$300-399 p.w.', 'Some Licence', 'N')
PROCESS to keep ('50-59', 'F', 'Negative Income', 'Some Licence', 'Y')
PROCESS to keep ('50-59', 'M', '$200-299 p.w.', 'Some Licence', 'Y')
PROCESS to keep ('90-99', 'M', '$800-999 p.w.', 'Some Licence', 'N')
PROCESS to keep ('20-29', 'M', '$400-599 p.w.', 'Some Licence', 'N')
PROCESS to keep ('40-49', 'M', '$400-599 p.w.', 'Some Licence', 'Y')
PROCESS to keep ('60-69', 'F', '$1250-1499 p.w

In [20]:
dict_hhid = dict(zip(hh_df["hhid"], hh_df[geo_lev]))
pop_rela_base[geo_lev] = pop_rela_base.apply(lambda r: dict_hhid[int(r["hhid"])],axis=1)
cols_main = [f"{x}_main" for x in PP_ATTS if x not in["relationship", "persid", "hhid", geo_lev]]
rename_cols = {f"{name}_{rela}": name for name in PP_ATTS if name not in["relationship", "persid", "hhid", geo_lev]}
pop_rela_base = pop_rela_base.drop(columns=cols_main)
pop_rela_base = pop_rela_base.rename(columns=rename_cols)
pop_rela_base

,age,sex,persinc,nolicence,anywork,hhid,relationship,POA
2485955,20-29,M,$1-199 p.w.,Some Licence,N,1329886,Grandchild,3022
8748470,10-19,F,$600-799 p.w.,Some Licence,N,1329886,Grandchild,3022
1229445,0-9,F,Zero Income,No Licence,N,1329886,Grandchild,3022
3146438,0-9,M,Zero Income,No Licence,N,1336566,Grandchild,3217
62647,10-19,M,Zero Income,No Licence,N,1336566,Grandchild,3217
...,...,...,...,...,...,...,...,...
1907924,20-29,M,$1-199 p.w.,Some Licence,N,1417685,Grandchild,3217
2566068,0-9,F,Zero Income,No Licence,N,1311618,Grandchild,3340
4094024,0-9,F,Zero Income,No Licence,N,1311618,Grandchild,3340
2202793,10-19,M,$1-199 p.w.,Some Licence,N,1232759,Grandchild,3076


## Method 2 let's check

In [13]:
input_pool_new = dict_pool_sample[rela].value_counts().reset_index()
to_del_df, pop_rela_check = process_rela_using_count(main_pp_df_all, rela, input_pool_new, geo_lev)

In [25]:
pop_rela_check

,hhid,age,sex,persinc,nolicence,anywork,POA,relationship
0,1329886,20-29,F,$1-199 p.w.,Some Licence,Y,3022,Grandchild
0,1329886,20-29,F,$1-199 p.w.,Some Licence,Y,3022,Grandchild
0,1329886,20-29,F,$1-199 p.w.,Some Licence,Y,3022,Grandchild
0,1336566,20-29,F,$1-199 p.w.,Some Licence,Y,3217,Grandchild
0,1336566,20-29,F,$1-199 p.w.,Some Licence,Y,3217,Grandchild
...,...,...,...,...,...,...,...,...
355,1417685,0-9,F,Zero Income,No Licence,N,3217,Grandchild
356,1311618,10-19,F,Zero Income,No Licence,Y,3340,Grandchild
356,1311618,10-19,F,Zero Income,No Licence,Y,3340,Grandchild
357,1232759,10-19,M,Zero Income,No Licence,N,3076,Grandchild


## Compare with VISTA

In [32]:
ls_atts_pp = [x for x in pop_rela_base.columns if x not in ["relationship", "POA", "hhid"]]
atts_labels = {att: list(pop_rela_base[att].unique()) for att in ls_atts_pp}
atts_labels

{'age': ['20-29',
  '10-19',
  '0-9',
  '30-39',
  '100+',
  '40-49',
  '50-59',
  '60-69',
  '80-89',
  '70-79',
  '90-99'],
 'sex': ['M', 'F'],
 'persinc': ['$1-199 p.w.',
  '$600-799 p.w.',
  'Zero Income',
  '$1000-1249 p.w.',
  '$200-299 p.w.',
  'Negative Income',
  '$1500-1999 p.w.',
  '$1250-1499 p.w.',
  '$300-399 p.w.',
  '$800-999 p.w.',
  '$400-599 p.w.',
  '$2000+ p.w.',
  'Missing/Refused'],
 'nolicence': ['Some Licence', 'No Licence'],
 'anywork': ['N', 'Y']}

In [33]:
def plot_bar_syn_rela(rela, att, syn_pp_base, syn_pp_check):
    df_base = syn_pp_base[syn_pp_base["relationship"] == rela]
    df_check = syn_pp_check[syn_pp_check["relationship"] == rela]
    n_base = len(df_base) 
    counts_base = df_base[att].value_counts()
    n_check = len(df_check) 
    counts_check = df_check[att].value_counts()
    data_base = []
    data_check = []
    data_sample = []
    if rela != "Main":
        df_rela_sample = pd.read_csv(os.path.join(processed_data, f"connect_main_{rela}.csv"))
        sub_df_sample = df_rela_sample[f"{att}_{rela}"].value_counts()
    else:
        df_rela_sample = pd.read_csv(os.path.join(processed_data, f"connect_hh_main.csv"))
        sub_df_sample = df_rela_sample[att].value_counts()
    n_sample = len(df_rela_sample)
    sub_df_sample = sub_df_sample / n_sample
    for state in atts_labels[att]:
        if state in counts_base.index:
            data_base.append((counts_base.at[state]/n_base)*100) 
        else:
            data_base.append(0) 

        if state in counts_check.index:
            data_check.append((counts_check.at[state]/n_check)*100) 
        else:
            data_check.append(0) 

        if state in sub_df_sample.index:
            data_sample.append((sub_df_sample[state])*100)
        else:
            data_sample.append(0)
    return {
        "vista": data_sample,
        "base": data_base,
        "check": data_check,
        "states": list(atts_labels[att])
    }

hold_dict = {}
for att in atts_labels:
    re_dict = plot_bar_syn_rela(att=att, rela="Grandchild", syn_pp_base=pop_rela_base, syn_pp_check=pop_rela_check)
    hold_dict[f"{rela}_{att}"] = re_dict

In [35]:
from scipy.spatial.distance import jensenshannon
d_df = []
ls_names = []
for key, d in hold_dict.items():
    score_base = jensenshannon(d["vista"], d["base"], base=2)
    score_check = jensenshannon(d["vista"], d["check"], base=2)
    d_df.append([score_base, score_check])
    ls_names.append(key)

cols_names = ["Base_score", "Check_score"]
df_all_scores = pd.DataFrame(d_df, index=ls_names, columns=cols_names)
df_all_scores    

,Base_score,Check_score
Grandchild_age,0.053485,0.219355
Grandchild_sex,0.002029,0.021309
Grandchild_persinc,0.046217,0.222212
Grandchild_nolicence,0.054730,0.276674
Grandchild_anywork,0.040967,0.132879


In [ ]:
d_rela = []
for rela in ls_rela:
    sub_df_avg = df_all_scores[df_all_scores.index.str.contains(rela)].mean()
    d_rela.append(list(sub_df_avg))

df_by_rela = pd.DataFrame(d_rela, columns=cols_names, index=ls_rela)
df_by_rela

In [2]:
with open(os.path.join(processed_data, 'dict_pool_sample.pickle'), 'rb') as handle:
    dict_pool_sample = pickle.load(handle)

dict_pool_sample.keys()

dict_keys(['Spouse', 'Child', 'Grandchild', 'Others', 'Main'])